Pacotes

In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup 
import requests 
import re 

Saúde

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}
urlhealth = "https://edition.cnn.com/health"
response = requests.get(urlhealth, headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

links = soup.select('a')
links = [link.get('href') for link in links]

filtered_links = [link for link in links if link and re.match(r'^/\d+', link) is not None] # pegando a continuação dos links
filtered_links = set(filtered_links)
urlmain = "https://edition.cnn.com"

saude = []
for link in filtered_links:
    newurl = urlmain + link # concatenando os links
    responsenew = requests.get(newurl, headers=headers) #acessando o link criado
    htmlnew = responsenew.text
    soupnew = BeautifulSoup(htmlnew, 'html.parser')
    titulo = soupnew.select_one('.headline__text')
    titulo = titulo.text.strip()
    autor_element= soupnew.select_one('.byline__name')
    autor = autor_element.text if autor_element else "Não identificado"
    categoria = re.search(r'/\d+/\d+/\d+/(\w+)',link)
    categoria = categoria.group(1)
    saude.append([titulo,autor,categoria])
saude = pd.DataFrame(saude, columns=["Titulo","Autor","Categoria"])
saude

In [ ]:
responsemain = requests.get(urlmain, headers=headers)
htmlmain = responsemain.text
soupmain = BeautifulSoup(htmlmain, 'html.parser')
categoriaslinks = soupmain.select("#pageHeader div div div.header__subnav div nav.subnav ul li:nth-child(1) a")
categoriaslinks = [categoria.get("href") for categoria in categoriaslinks]

noticias = []

for link in categoriaslinks:
    responsecat = requests.get(link, headers=headers)
    htmlcat = responsecat.text
    soupcat = BeautifulSoup(htmlcat, 'html.parser')

    linksnoticias = soupcat.select("a")
    linksnoticias = [linksn.get("href") for linksn in linksnoticias]
    filtered_linksnoticias = [linkn for linkn in linksnoticias if linkn and re.match(r'^/\d+', linkn) is not None]
    filtered_linksnoticias = set(filtered_linksnoticias)

    for linkfiltrado in filtered_linksnoticias:
        urlnoticias = urlmain + linkfiltrado
        responsenoticias = requests.get(urlnoticias, headers=headers)
        htmlnoticias = responsenoticias.text
        soupnoticias = BeautifulSoup(htmlnoticias, 'html.parser')

        titulonoticias = soupnoticias.select_one('.headline__text')
        if titulonoticias:
            titulonoticias = titulonoticias.text.strip()
        else:
            titulonoticias = "Não identificado"
        autor_elementnoticias= soupnoticias.select_one('.byline__name')
        autornoticias = autor_elementnoticias.text if autor_elementnoticias else "Não identificado"
        categorianoticias = re.search(r'/\d+/\d+/\d+/(\w+)',linkfiltrado)
        categorianoticias = categorianoticias.group(1)
        noticias.append([titulonoticias,autornoticias,categorianoticias])

noticias = pd.DataFrame(noticias, columns=["Titulo","Autor","Categoria"])
noticias = noticias.drop_duplicates()
noticias